# Monday, May 1st, 2023

In [2]:
import requests
import re

In [3]:
page = requests.get('https://www.gutenberg.org/cache/epub/70635/pg70635.txt')

In [4]:
text = page.text

In [7]:
print(text[3000:4000])

ters
pertaining to print, is at least encouraging, and this leads me to say
that of my earlier plays there have been sold in paper covers _three
hundred and twenty thousand copies_, besides an edition in cloth.

                                                               THE AUTHOR.

Chicago, July 11, 1895.




                               TOPP’S TWINS

                       A FARCE-COMEDY IN FOUR ACTS

                             BY T. S. DENISON

                               _Author of_
  Odds with the Enemy, Initiating a Granger, Wanted, a Correspondent, A
    Family Strike, Seth Greenback, Louva, the Pauper, Hans Von Smash,
 Borrowing Trouble, Two Ghosts in White, The Pull-Back, Country Justice,
        The Assessor, The Sparkling Cup, Our Country, Irish Linen
     Peddler, The School Ma’am, Kansas Immigrants, An Only Daughter,
         Too Much of a Good Thing, Under the Laurels, Hard Cider,
         The Danger Signal, Wide Enough for Two, Pets of


The `^` is used as a logical `not` in RegEx:

In [13]:
pattern = r'\b[oO][\w]*s\b'
matches = re.findall(pattern,text)
matches[:10]

['Outlines',
 'Odds',
 'odious',
 'oysters',
 'outrageous',
 'opens',
 'opens',
 'outrageous',
 'opens',
 'ourselves']

Let's find all words that start with `o` (or `O`), do not contain the letter `p`, and end with `s`:

In [16]:
pattern = r'\b[oO][^p^\W]*s\b'
matches = re.findall(pattern,text)
matches[:10]

['Outlines',
 'Odds',
 'odious',
 'oysters',
 'outrageous',
 'outrageous',
 'ourselves',
 'oysters',
 'obstacles',
 'obtuseness']

In [28]:
pattern = r'\b[oO][^\W]*s\b'
matches = re.findall(pattern,text)
matches[:10]

['Outlines',
 'Odds',
 'odious',
 'oysters',
 'outrageous',
 'opens',
 'opens',
 'outrageous',
 'opens',
 'ourselves']

We can also specify ranges of acceptable characters (say numbers or letters):

Let's find all words that start with a letter between `o` and `r` (or `O` and `R`), end with a letter between `a` and `g`, with at least 3 letters in between the starting and ending letter.

In [30]:
pattern = r'\b[o-rO-R]\w{3,}[a-g]\b'
matches = re.findall(pattern,text)
matches[:10]

['people',
 'online',
 'people',
 'Release',
 'Produced',
 'Online',
 'Proofreading',
 'produced',
 'Rejected',
 'requisite']

## Project 8 - Address book

In [31]:
directory_url = 'https://pharmacy.buffalo.edu/faculty-staff.html'

In [32]:
directory_page = requests.get(directory_url)

In [33]:
directory_text = directory_page.text

In [36]:
print(directory_text[:1000])

<!DOCTYPE HTML><html lang="en" class="ubcms-65"><head><link rel="preconnect" href="https://www.googletagmanager.com/" crossorigin/><link rel="dns-prefetch" href="https://www.googletagmanager.com/"/><link rel="dns-prefetch" href="https://connect.facebook.net/"/><link rel="dns-prefetch" href="https://www.google-analytics.com/"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta http-equiv="content-type" content="text/html; charset=UTF-8"/><meta id="meta-viewport" name="viewport" content="width=device-width,initial-scale=1"/><script>if (screen.width > 720 && screen.width < 960) document.getElementById('meta-viewport').setAttribute('content','width=960');</script><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src='https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);})(window,document,'script','data

Looking directly at the page text, it's very difficult to make sense of anything. One thing that will be really helpful is to replace the string `>` with `>\n`.

In [37]:
directory_text = directory_text.replace('>','>\n')

In [40]:
print(directory_text)

<!DOCTYPE HTML>
<html lang="en" class="ubcms-65">
<head>
<link rel="preconnect" href="https://www.googletagmanager.com/" crossorigin/>
<link rel="dns-prefetch" href="https://www.googletagmanager.com/"/>
<link rel="dns-prefetch" href="https://connect.facebook.net/"/>
<link rel="dns-prefetch" href="https://www.google-analytics.com/"/>
<meta http-equiv="X-UA-Compatible" content="IE=edge"/>
<meta http-equiv="content-type" content="text/html; charset=UTF-8"/>
<meta id="meta-viewport" name="viewport" content="width=device-width,initial-scale=1"/>
<script>
if (screen.width >
 720 && screen.width < 960) document.getElementById('meta-viewport').setAttribute('content','width=960');</script>
<script>
(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src='https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);})(window,document,

After searching for `Adams, Kimberly`, we want to write a regex pattern that could match something like:

`<a class="facultyindexname" href="https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=kadams4">`

In [45]:
url_pattern = r'https?://.*\b'
url_matches = re.findall(url_pattern,directory_text)

In [48]:
url_matches[:30]

['https://www.googletagmanager.com/" crossorigin',
 'https://www.googletagmanager.com',
 'https://connect.facebook.net',
 'https://www.google-analytics.com',
 "https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);})(window,document,'script','dataLayer','GTM-T5KRRKT');</script",
 'https://pharmacy.buffalo.edu/faculty-staff.html',
 'https://pharmacy.buffalo.edu/_jcr_content/social.img.512.auto.jpg/1583336724209.jpg',
 'https://www.googletagmanager.com/ns.html?id=GTM-T5KRRKT" height="0" width="0" style="display:none;visibility:hidden',
 'http://pharmacy.buffalo.edu',
 'https://www.facebook.com/UBSoPPS/" class="button facebook',
 'https://twitter.com/UBPharm" class="button twitter',
 'https://www.instagram.com/ubpharm" class="button instagram',
 'https://www.linkedin.com/school/state-university-of-new-york-at-buffalo---school-of-pharmacy-and-pharmaceutical-sciences/" class="button linkedin',
 'https://www.youtube.com/user/UBPharmacy" class="button youtube',
 'ht

Recall: RegEx will give special meaning to `.` and `?`. We can escape these special characters using `\.` and `\?`:

In [51]:
url_pattern = r'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile\.html\?ubit=\w+\b'
url_matches = re.findall(url_pattern,directory_text)

In [54]:
url_matches

['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=kadams4',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=adolf',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=npaolini',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=attwood3',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=jb',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=svb',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=bsbasile',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=eb',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=amabile',
 'https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=benning2',
 'https://pharmacy.buffalo.edu/c

In [56]:
person_url = url_matches[0]

In [59]:
person_page = requests.get(person_url)
person_text = person_page.text
person_text = person_text.replace('>','>\n')

In [61]:
print(person_text)

<!DOCTYPE HTML>
<html lang="en" class="ubcms-65">
<head>
<link rel="preconnect" href="https://www.googletagmanager.com/" crossorigin/>
<link rel="dns-prefetch" href="https://www.googletagmanager.com/"/>
<link rel="dns-prefetch" href="https://connect.facebook.net/"/>
<link rel="dns-prefetch" href="https://www.google-analytics.com/"/>
<meta http-equiv="X-UA-Compatible" content="IE=edge"/>
<meta http-equiv="content-type" content="text/html; charset=UTF-8"/>
<meta id="meta-viewport" name="viewport" content="width=device-width,initial-scale=1"/>
<script>
if (screen.width >
 720 && screen.width < 960) document.getElementById('meta-viewport').setAttribute('content','width=960');</script>
<script>
(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src='https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);})(window,document,

Looking through the page source, it looks like the block of contact information is preceded by the following HTML:
`<div class="facultyprofilecontact">`

Let's try splitting our page source on this text, then take the index `1` element of the split:

In [69]:
#split1 = person_text.split('<div class="facultyprofilecontact">')[1]
split1 = person_text.split('<span id="lblAcademicOfficeAddressMobile">')[1]

print(split1[:1000])


117 Pharmacy Building<br/>
** South Campus **<br/>
Buffalo, NY <br/>
Phone: (716) 645-4833 <br/>
<a href="mailto:kadams4@buffalo.edu">
kadams4@buffalo.edu</a>
<br/>
<div class="facultyprofilemapicon">
<a href="http://maps.google.com/maps?q=117+Pharmacy+Building+**+South+Campus+**+Buffalo+NY+" target="_blank">
View map</a>
</div>
</span>
</div>
<br/>
<div class="facultyprofile-expandablecontent">
</div>
<br/>
</div>
</div>
</div>
</div>
<div class="facultyprofilebreaker">
<hr class="facultyprofilehorizonalbreaker"/>
</div>
<div class="facultyprofilecontents">
<hr style="visibility: hidden; margin: 2px 0px 1px 0px;"/>
</div>
<a href='#' class='scrollup'>
</a>
</div>
<div class="facultyprofilecontact">
<div class="title facultyprofile-hide-in-mobile">
<div class="title">
<h3>
Contact Information</h3>
 <span id="lblAcademicOfficeAddress">
117 Pharmacy Building<br/>
** South Campus **<br/>
Buffalo, NY <br/>
Phone: (716) 645-4833 <br/>
<a href="mailto:kadams4@buffalo.edu">
kadams4@buffalo.e

We can then split the string on some substring that appears after the contact information, and keep the index `0` part:

In [70]:
split2 = split1.split('<div class="facultyprofilemapicon">')[0]

In [71]:
print(split2)


117 Pharmacy Building<br/>
** South Campus **<br/>
Buffalo, NY <br/>
Phone: (716) 645-4833 <br/>
<a href="mailto:kadams4@buffalo.edu">
kadams4@buffalo.edu</a>
<br/>



Let's make a function that does these splits:

In [72]:
def get_contact_block(person_text):
    split1 = person_text.split('<span id="lblAcademicOfficeAddressMobile">')[1]
    split2 = split1.split('<div class="facultyprofilemapicon">')[0]
    return split2

In [74]:
print(get_contact_block(person_text))


117 Pharmacy Building<br/>
** South Campus **<br/>
Buffalo, NY <br/>
Phone: (716) 645-4833 <br/>
<a href="mailto:kadams4@buffalo.edu">
kadams4@buffalo.edu</a>
<br/>



Let's test it for other pages:

In [80]:
contact_blocks = []
person_texts = []

for person_url in url_matches:
    person_page = requests.get(person_url)
    person_text = person_page.text
    person_text = person_text.replace('>','>\n')
    
    contact_block = get_contact_block(person_text)
    
    contact_blocks.append(contact_block)
    person_texts.append(person_text)
    
    print(person_url)
    print(contact_block)

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=kadams4

117 Pharmacy Building<br/>
** South Campus **<br/>
Buffalo, NY <br/>
Phone: (716) 645-4833 <br/>
<a href="mailto:kadams4@buffalo.edu">
kadams4@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=adolf

453 Pharmacy Building<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-6112 <br/>
<a href="mailto:adolf@buffalo.edu">
adolf@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=npaolini

209 Pharmacy Building<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-3915 <br/>
<a href="mailto:npaolini@buffalo.edu">
npaolini@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=attwood3

<br/>
<a href="mailto:attwood3@buffalo.edu">
attwood3@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profi

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=matthewc

313 Abbott Hall<br/>
South Campus<br/>
Buffalo, NY 14214 <br/>
Phone: (716) 645-4804 <br/>
<a href="mailto:matthewc@buffalo.edu">
matthewc@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=cjdaly

204 Pharmacy Building<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-4793 <br/>
<a href="mailto:cjdaly@buffalo.edu">
cjdaly@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=sldavis3

129 Cooke Hall<br/>
** North Campus **<br/>
Buffalo, NY <br/>
Phone: (716) 645-2498 <br/>
<a href="mailto:sldavis3@buffalo.edu">
sldavis3@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=delarosa

156 Pharmacy Building<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-2259 <br/>
Fax: (716) 645-3688 <br/>
<a href="mailto:delarosa@buffalo.e

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=lehill

Center Of Excellence<br/>
701 Ellicott Street<br/>
Buffalo, NY <br/>
Phone: (716) 645-2823 <br/>
<a href="mailto:lehill@buffalo.edu">
lehill@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=cchoefer

<br/>
<a href="mailto:cchoefer@buffalo.edu">
cchoefer@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=pnholden

Buffalo, NY 14214-8033 <br/>
Phone: (716) 881-8917 <br/>
<a href="mailto:pnholden@buffalo.edu">
pnholden@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=rhuang

126<br/>
<br/>
Phone: (716) 645-2825 <br/>
<a href="mailto:rhuang@buffalo.edu">
rhuang@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=shihanhu

<br/>
<a href="mailto:shihanhu@buffalo.edu">


https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=dmager

431 Pharmacy Building<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-2903 <br/>
Fax: (716) 829-6569 <br/>
<a href="mailto:dmager@buffalo.edu">
dmager@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=cmaponga

213 Pharmacy Building<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-4793 <br/>
<a href="mailto:cmaponga@buffalo.edu">
cmaponga@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=yaelmark

<br/>
<a href="mailto:yaelmark@buffalo.edu">
yaelmark@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=cjmeaney

203 Pharmacy Building<br/>
Buffalo, NY 14214 <br/>
Phone: 716-645-2826 <br/>
Fax: 716-829-6093 <br/>
<a href="mailto:cjmeaney@buffalo.edu">
cjmeaney@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=saileesu

<br/>
<a href="mailto:saileesu@buffalo.edu">
saileesu@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=irenehon

340 Abbott<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-1540 <br/>
Fax: (716) 829-3109 <br/>
<a href="mailto:irenehon@buffalo.edu">
irenehon@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=tren

126 Cooke Hall<br/>
** North Campus **<br/>
Buffalo, NY <br/>
<a href="mailto:tren@buffalo.edu">
tren@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=maryried

221 Pharmacy Building<br/>
**South Campus**<br/>
Buffalo, NY 14214 <br/>
Phone: 716-829-3530 <br/>
<a href="mailto:maryried@buffalo.edu">
maryried@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.ht

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=wwu35

<br/>
<a href="mailto:wwu35@buffalo.edu">
wwu35@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=yxu37

<br/>
<a href="mailto:yxu37@buffalo.edu">
yxu37@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=chaoxue

445<br/>
<br/>
Phone: (716) 645-4833 <br/>
<a href="mailto:chaoxue@buffalo.edu">
chaoxue@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=ltyates

1114 Kimball Hall, South Campus<br/>
Buffalo, NY <br/>
Phone: (716) 645-3432 <br/>
Fax: (716) 829-6067 <br/>
<a href="mailto:ltyates@buffalo.edu">
ltyates@buffalo.edu</a>
<br/>

https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=yjyou

630 Pharmacy Building<br/>
Buffalo, NY 14214-8033 <br/>
Phone: (716) 645-4843 <br/>
<a href="mailto